In [2]:
import requests
from selenium import webdriver
from webdriver_manager.utils import chrome_version
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
# これをimportしておくことで自分でcheromedriverをインストールする必要がない
import chromedriver_binary
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
op = Options()
op.add_argument("no-sandbox")
op.add_argument("--disable-extensions")
# 動作確認のときはoffにする
# op.add_argument("--headless")
op.add_argument('--disable-gpu')  
op.add_argument('--ignore-certificate-errors')
op.add_argument('--allow-running-insecure-content')
op.add_argument('--disable-web-security')
op.add_argument('--disable-desktop-notifications')
op.add_argument("--disable-extensions")
op.add_argument('--lang=ja')
# 画像を保存したいときはoffにする
op.add_argument('--blink-settings=imagesEnabled=false')
op.add_argument('--disable-dev-shm-usage')
op.add_argument('--proxy-server="direct://"')
op.add_argument('--proxy-bypass-list=*')    
op.add_argument('--start-maximized')

driver_path =  "https://zenn.dev/articles?page=1"

dates = []
titles  = []
urls = []
usernames = []

In [3]:
#ユーザー情報の取得


driver = webdriver.Chrome(chrome_options= op)
# 初期のurlに移行できるか確認(ここで開かれるwindowをインスタンスwindowという)
driver.get(driver_path)

#urlの読み込みに時間がかかる可能性があるため
time.sleep(5)

# 1ページあたり42記事
kiji_containers = driver.find_elements_by_class_name("ArticleList_itemContainer__1ECL5")

date = kiji_containers[0].find_element_by_tag_name("time").get_attribute("datetime")
title = kiji_containers[0].find_element_by_class_name("ArticleList_title__3KqyY").text
url = kiji_containers[0].find_element_by_class_name("ArticleList_emoji__1RiRB").get_attribute("href")
username = kiji_containers[0].find_element_by_class_name("ArticleList_userName__3MtlK").text
pro = kiji_containers[0].find_element_by_class_name("ArticleList_user__1XkRB").get_attribute("href")
like = kiji_containers[4].find_element_by_class_name("ArticleList_like__3OVME").text

pro

# ページネーションの部分
next_= driver.find_element_by_class_name("ListPagination_linkNext__2-ejg")
# prev = driver.find_element_by_class_name("ListPagination_linkBack__1PhXo")

try:
    next_.click()
#要素がなくなったら最後のページなので処理を終わる
except:
    judge=False

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [4]:
df_users = pd.read_csv("userinfo_from_article.csv")["ユーザーURL"]

In [6]:
len(df_users)

6951

In [12]:
# driverのセットアップ
driver = webdriver.Chrome(chrome_options=op)

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [8]:
# 初期のurlに移行できるか確認(ここで開かれるwindowをインスタンスwindowという)
driver.get(driver_path)

#urlの読み込みに時間がかかる可能性があるため
time.sleep(5)

In [9]:
descriptions = []
all_likes = []
followers = []
githubs = []
twitters = []
my_urls = []

In [81]:

#ない可能性
discription = driver.find_element_by_class_name("link-text-inside").text

all_like=driver.find_element_by_xpath('//button[contains(text(), "Likes")]').text.replace("\nLikes", "")
follower = driver.find_element_by_xpath('//button[contains(text(), "Followers")]').text.replace("\nFollowers", "")


github = driver.find_element_by_xpath('//a[contains(@href, "github")]').get_attribute("href")
twitter = driver.find_element_by_xpath('//a[contains(@href, "twitter")]').get_attribute("href")
my_url = driver.find_element_by_class_name('tooltip-container').get_attribute("href")

article_num = driver.find_element_by_xpath('//a[contains(text(), "Articles")]').text.replace("Articles", "")
scrap_num = driver.find_element_by_xpath('//a[contains(text(), "Scraps")]').text.replace("Scraps", "")

In [82]:
scrap_num 

'0'

In [13]:
#df_usersは userのurlのシリーズ
def profile_get(driver, df_users):
    users = []
    descriptions = []
    all_likes = []
    followers = []
    githubs = []
    twitters = []
    my_urls = []
    article_nums = []
    scrap_nums  = []
    
    user_num = 1
    
    try:
        for df_user in df_users:
            print("{}/{}を進行中".format(user_num, len(df_users)))
            
            #　ユーザーページヘジャンプ
            driver.get(df_user)
            time.sleep(0.5)

            # descriptionについて
            try:
                description = driver.find_element_by_class_name("link-text-inside").text
            except:
                description = "なし"

            # すべての記事のいいね数について
            try:
                all_like = driver.find_element_by_xpath('//button[contains(text(), "Likes")]').text.replace("\nLikes", "")
            except:
                all_like = "0"

            # フォロワーについて
            try:
                follower = driver.find_element_by_xpath('//button[contains(text(), "Followers")]').text.replace("\nFollowers", "")
            except:
                follower = "0"

            # githubについて
            try:
                github = driver.find_element_by_xpath('//a[contains(@href, "github")]').get_attribute("href")
            except:
                github = "none"

            # twitterについて
            try:
                twitter = driver.find_element_by_xpath('//a[contains(@href, "twitter")]').get_attribute("href")
            except:
                twitter = "none"

            # 自分のリンクについて
            try:
                my_url = driver.find_element_by_class_name('tooltip-container').get_attribute("href")
            except:
                my_url = "none"

            # articleの数
            try:
                article_num = driver.find_element_by_xpath('//a[contains(text(), "Articles")]').text.replace("Articles", "")
            except:
                article_num = "0"

            # scrapsの数
            try:
                scrap_num = driver.find_element_by_xpath('//a[contains(text(), "Scraps")]').text.replace("Scraps", "")
            except:
                scrap_num = "0"

            users.append(df_user)
            descriptions.append(description)
            all_likes.append(all_like)
            followers.append(follower)
            githubs.append(github)
            twitters.append(twitter)
            my_urls.append(my_url)
            article_nums.append(article_num)
            scrap_nums.append(scrap_num)
            
            user_num+=1
            
    except KeyboardInterrupt:
        print("{}ユーザー目で中断されました".format(user_num))
    
    except:
        print("{}ユーザーめで何らかのエラーが発生しました".format(user_num))
        
    data_list = [users, descriptions, all_likes, followers, githubs,  twitters, my_urls,  article_nums, scrap_nums]
    return data_list, users, descriptions, all_likes, followers, githubs,  twitters, my_urls,  article_nums, scrap_nums
        

In [14]:
data_list, users, descriptions, all_likes, followers, githubs,  twitters, my_urls,  article_nums, scrap_nums=profile_get(driver, df_users)

1/6951を進行中
2/6951を進行中
3/6951を進行中
4/6951を進行中
5/6951を進行中
6/6951を進行中
7/6951を進行中
8/6951を進行中
9/6951を進行中
10/6951を進行中
11/6951を進行中
12/6951を進行中
13/6951を進行中
14/6951を進行中
15/6951を進行中
15ユーザー目で中断されました


In [16]:
len(data_list)

9

In [13]:
df

,ユーザーネーム,ユーザーURL,github,ポートフォリオ,TwitterURL,Twitterフォロー数,Twitterフォロワー数,DM可否
0,ゆーたろー,https://zenn.dev/shimotaroo,https://github.com/shimotaroo,https://linktr.ee/shimotaroo,https://twitter.com/shimotaroo,302,"3,688",可能
1,barusu,https://zenn.dev/barusu,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,817",不可
2,大野純,https://zenn.dev/j_09,https://github.com/junono09,none,https://twitter.com/jo51608309,84,87,可能
3,centerfield77,https://zenn.dev/centerfield77,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,817",不可
4,あわしろいくや,https://zenn.dev/ikuya,https://github.com/ikunya,https://blog.goo.ne.jp/ikunya/,https://twitter.com/ikunya,"2,762","2,506",不可
...,...,...,...,...,...,...,...,...
6946,suztomo,https://zenn.dev/suztomo,https://github.com/suztomo,none,https://twitter.com/suztomo,"1,128","1,513",不可
6947,Satoshi Kita,https://zenn.dev/kitar,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,818",不可
6948,ryo_kawamata,https://zenn.dev/ryo_kawamata,https://github.com/kawamataryo,https://qiita.com/ryo2132,https://twitter.com/kawamataryo,838,"1,495",可能
6949,shogo.yamada,https://zenn.dev/shogo,https://github.com/yshogo,https://note.com/shogoyamada,https://twitter.com/yshogo87,834,"2,199",可能


In [6]:
df = pd.read_csv("Zennリスト.csv").iloc[:,1:]

In [11]:
df2 = df[~df.duplicated(subset="ユーザーネーム")].reset_index(drop=True)

In [12]:
df2

,ユーザーネーム,ユーザーURL,github,ポートフォリオ,TwitterURL,Twitterフォロー数,Twitterフォロワー数,DM可否
0,ゆーたろー,https://zenn.dev/shimotaroo,https://github.com/shimotaroo,https://linktr.ee/shimotaroo,https://twitter.com/shimotaroo,302,"3,688",可能
1,barusu,https://zenn.dev/barusu,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,817",不可
2,大野純,https://zenn.dev/j_09,https://github.com/junono09,none,https://twitter.com/jo51608309,84,87,可能
3,centerfield77,https://zenn.dev/centerfield77,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,817",不可
4,あわしろいくや,https://zenn.dev/ikuya,https://github.com/ikunya,https://blog.goo.ne.jp/ikunya/,https://twitter.com/ikunya,"2,762","2,506",不可
...,...,...,...,...,...,...,...,...
2556,i_am_master_yoda,https://zenn.dev/i_am_master_yod,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,818",不可
2557,ゆうき,https://zenn.dev/yuuki,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,818",不可
2558,Uchida Tsutomu,https://zenn.dev/sidodtv,https://github.com/zenn-dev/zenn-roadmap/proje...,https://d-t-v.com/,https://twitter.com/sidodtv,"3,403","1,623",可能
2559,suztomo,https://zenn.dev/suztomo,https://github.com/suztomo,none,https://twitter.com/suztomo,"1,128","1,513",不可


In [14]:
df2.to_csv("Zennリスト.fin.csv")

In [15]:
df3 = pd.read_csv("Zennリスト.fin.csv")

In [16]:
df3

,Unnamed: 0,ユーザーネーム,ユーザーURL,github,ポートフォリオ,TwitterURL,Twitterフォロー数,Twitterフォロワー数,DM可否
0,0,ゆーたろー,https://zenn.dev/shimotaroo,https://github.com/shimotaroo,https://linktr.ee/shimotaroo,https://twitter.com/shimotaroo,302,"3,688",可能
1,1,barusu,https://zenn.dev/barusu,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,817",不可
2,2,大野純,https://zenn.dev/j_09,https://github.com/junono09,none,https://twitter.com/jo51608309,84,87,可能
3,3,centerfield77,https://zenn.dev/centerfield77,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,817",不可
4,4,あわしろいくや,https://zenn.dev/ikuya,https://github.com/ikunya,https://blog.goo.ne.jp/ikunya/,https://twitter.com/ikunya,"2,762","2,506",不可
...,...,...,...,...,...,...,...,...,...
2556,2556,i_am_master_yoda,https://zenn.dev/i_am_master_yod,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,818",不可
2557,2557,ゆうき,https://zenn.dev/yuuki,https://github.com/zenn-dev/zenn-roadmap/proje...,none,https://twitter.com/zenn_dev,0,"2,818",不可
2558,2558,Uchida Tsutomu,https://zenn.dev/sidodtv,https://github.com/zenn-dev/zenn-roadmap/proje...,https://d-t-v.com/,https://twitter.com/sidodtv,"3,403","1,623",可能
2559,2559,suztomo,https://zenn.dev/suztomo,https://github.com/suztomo,none,https://twitter.com/suztomo,"1,128","1,513",不可
